In [1]:
import sys
print sys.executable
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

/Users/lisa1010/tf_venv/bin/python


In [2]:
import tensorflow as tf
import tflearn
import numpy as np
import time

In [3]:
import concept_dependency_graph as cdg
import dataset_utils
import dynamics_model_class as dm
import data_generator as dgen
from filepaths import *
from constants import *

In [4]:
dataset_name = "10000stud_100seq_modulo"

In [5]:
# load the numpy matrices
input_data_, output_mask_, target_data_ = dataset_utils.load_rnn_data(dataset_name)

In [6]:
print input_data_.shape
print target_data_.shape

(10000, 100, 20)
(10000, 100, 10)


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
x_train, x_test, mask_train, mask_test, y_train, y_test = train_test_split(input_data_, output_mask_, target_data_, test_size=0.1, random_state=42)

In [9]:
train_data = (x_train, mask_train, y_train)

#### Build, train and save RNN Dynamics Model

In [10]:
import models_dict_utils

In [11]:
# Each RNN model can be identified by its model_id string. 
# We will save checkpoints separately for each model. 
# Models can have different architectures, parameter dimensions etc. and are specified in models_dict.json
model_id = "learned_from_modulo"

In [12]:
# Specify input / output dimensions and hidden size
n_timesteps = 100
n_inputdim = 20
n_outputdim = 10
n_hidden = 32

# If you are creating a new RNN model or just to check if it already exists:
# Only needs to be done once for each model

models_dict_utils.check_model_exists_or_create_new(model_id, n_inputdim, n_hidden, n_outputdim)

A model with the same model_id 'learned_from_modulo' already exists. 
No differences found. Yay! 


In [13]:
# Load model from latest checkpoint 
dmodel = dm.DynamicsModel(model_id=model_id, timesteps=100, load_checkpoint=True)

Loading RNN dynamics model...
Directory path for tensorboard summaries: ../tensorboard_logs/learned_from_modulo/
Checkpoint directory path: ../checkpoints/learned_from_modulo/
Checkpoint filename: /Users/lisa1010/dev/smart-tutor/checkpoints/learned_from_modulo/_-2286
Checkpoint loaded.
Model loaded.


In [14]:
# important to cast preds as numpy array. 
preds = np.array(dmodel.predict(x_test[:1]))

In [15]:
print preds.shape

(1, 100, 10)


In [16]:
print preds[0,0]

[ 0.3591972   0.43897602  0.34933782  0.27990732  0.27030045  0.2504656
  0.26783279  0.22879325  0.24082358  0.22392181]


In [17]:
# Load model with different number of timesteps from checkpoint
# Since RNN weights don't depend on # timesteps (weights are the same across time), we can load in the weights for 
# any number of timesteps. The timesteps parameter describes the # of timesteps in the input data.
generator_model = dm.DynamicsModel(model_id=model_id, timesteps=1, load_checkpoint=True)

Loading RNN dynamics model...
Directory path for tensorboard summaries: ../tensorboard_logs/learned_from_modulo/
Checkpoint directory path: ../checkpoints/learned_from_modulo/
Checkpoint filename: /Users/lisa1010/dev/smart-tutor/checkpoints/learned_from_modulo/_-2286
Checkpoint loaded.
Model loaded.


In [18]:
# make a prediction
preds = generator_model.predict(x_test[:1,:1, :])

In [19]:
print preds[0][0]

[0.359197199344635, 0.4389760196208954, 0.3493378162384033, 0.27990731596946716, 0.2703004479408264, 0.25046560168266296, 0.26783278584480286, 0.22879324853420258, 0.24082358181476593, 0.22392180562019348]
